In [12]:
import math

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

train_data = pd.read_csv('dataset/price_data_tr.csv')
heldout_data = pd.read_csv('dataset/price_data_val.csv')
test_data = pd.read_csv('dataset/price_data_ts.csv')

In [13]:
train_data = train_data.drop(['id','date'], axis=1)

In [14]:
nor_train = (train_data - train_data.mean()) / train_data.std()

In [17]:
np.abs(nor_train) > 3

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12963,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12964,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12965,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12966,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [16]:
train_data.apply(lambda x: (x-x.mean()) / x.std()).describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04,1.296800e+04
mean,1.194466e-16,-2.465640e-16,-9.424225e-17,2.257431e-16,1.808136e-17,-1.534176e-17,1.643760e-17,4.438152e-17,-1.786219e-16,3.364229e-16,-1.260216e-16,7.287337e-17,-1.244874e-15,-1.150632e-17,8.554347e-14,-7.708687e-15,1.051968e-13,7.506505e-17,2.520432e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.267291e+00,-3.723418e+00,-2.750481e+00,-1.864920e+00,-3.457666e-01,-9.234253e-01,-8.815110e-02,-3.043774e-01,-3.752838e+00,-3.981450e+00,-1.717474e+00,-6.636124e-01,-2.431368e+00,-2.110369e-01,-1.442874e+00,-2.912130e+00,-2.128667e+00,-2.235679e+00,-4.329467e-01
25%,-5.906118e-01,-4.083353e-01,-4.777087e-01,-7.170270e-01,-2.383347e-01,-9.234253e-01,-8.815110e-02,-3.043774e-01,-6.303689e-01,-5.630546e-01,-7.235255e-01,-6.636124e-01,-6.540992e-01,-2.110369e-01,-8.450199e-01,-6.427608e-01,-8.117650e-01,-7.267534e-01,-2.749471e-01
50%,-2.451586e-01,-4.083353e-01,1.716547e-01,-1.837454e-01,-1.765637e-01,4.651836e-03,-8.815110e-02,-3.043774e-01,-6.303689e-01,-5.630546e-01,-2.750368e-01,-6.636124e-01,1.320006e-01,-2.110369e-01,-2.471658e-01,9.172740e-02,-1.179133e-01,-2.140115e-01,-1.850962e-01
75%,2.797204e-01,6.966923e-01,4.963363e-01,5.040078e-01,-1.049524e-01,9.327290e-01,-8.815110e-02,-3.043774e-01,9.308657e-01,2.915442e-01,5.128488e-01,5.888635e-01,8.839222e-01,-2.110369e-01,7.430302e-01,8.498042e-01,6.325792e-01,5.477763e-01,-9.871410e-02
max,1.802201e+01,7.326858e+00,7.639333e+00,1.251240e+01,3.934699e+01,3.716960e+00,1.134328e+01,4.916742e+00,2.492100e+00,4.564538e+00,9.240197e+00,8.741343e+00,1.499131e+00,4.786438e+00,2.256349e+00,1.565746e+00,6.367475e+00,6.041439e+00,3.053870e+01
